In [1]:
import os, shutil, gzip
import csv

# PULPy Helper

This notebook is intended to help streamline working with PULpy. Using it you can:

* rename input files according to PULpy's requirements
* move input files from a provided directory to PULpy's input subdirectories
* clear the PULpy working directory of input and output files
* export input and output files from the PULpy working directory
* run PULpy

### Directories
Provide the notebook with your PULpy working directory and the paths to your input and output directories, if relevant.

In [2]:
pulpyd = '/home/ecutts/PULpy/' #absolute path to PULpy directory
idir = '/home/ecutts/Bacteroidetes/MAGs/' # absolute path to input directory
odir = '' # absolute path to output directory

# PULpy-prep

Run the following cells to prepare a directory of input files for analysis by PULpy. The files will be copied to the appropriate PULpy input directories and renamed according to PULpy's naming conventions. You can choose to clear pre-existing input and output files within the PULpy working directory or to simply add your input files. The program identifies files to move by looking for .fna or .fna.gz files, which are required by PULpy even if using annotations.

### Input directory requirements
Your input directory should be either (1) a directory of directories, with each directory containing the files associated with a single genome or (2) a flat directory containing all of the files to be input. 

### Clear PULpy working directory first?
Set `clear = True` if you would like all input and output files in the PULpy working directory to be deleted before adding your inputs.

In [3]:
clear = True # change to True if you wish to clear the PULpy working directory before import

def cleardir(d):
    for filename in os.listdir(d):
        filepath = os.path.join(d, filename)
        os.remove(filepath)

def clearPULpy(pulpyd):
    clear_dirs = ['dbcan', 'pfam', 'feature_table', 'genomes', 'proteins', 'puls']
    os.chdir(pulpyd)
    for d in os.listdir():
        if d in clear_dirs:
            cleardir(d)

if clear: #if clear is True, then the PULpy output directories will be cleared
    clearPULpy(pulpyd)

In [6]:
os.chdir(idir)

dd = False # Is the directory a directory of directories?
for f in os.listdir():
    if not os.path.isfile(f):
        dd = True

def find_fnas(d):
    fnas = {}
    for f in os.listdir(d):
        split = f.split('.')[1] # splits used because IMG downloads use ".genes.fna" as well as ".fna"
        if f.endswith('.fna.gz'): # favor .fna.gz files over .fna
            idd = f.split('.fna.gz')[0]
            fnas[idd] = f
        elif split == 'fna':
            idd = f.split('.fna')[0]
            fnas[idd] = f
    return fnas
        
def find_files(d, ids):
    gffs = {}
    faas = {}
    for f in os.listdir(d):
        idd = f.split('.')[0]
        if idd in ids:
            if f.endswith('.gff'):
                gffs[idd] = f
            elif f.endswith('.faa'): #IMG protein fastas end in .genes.faa, this may be changed for others
                faas[idd] = f
    return gffs, faas


# get list of files to move (relative paths to idir)
if dd == False:
    fnas = find_fnas(idir)
    ids = fnas.keys()
    files = dict.fromkeys(ids)
    found  = find_files(idir, ids)
    for idd in files.keys():
        files[idd] = [fnas[idd], found[0][idd],found[1][idd]]
    
else:
    files = {}
    
    for d in os.listdir():
        fna = find_fnas(d)
        idd = list(fna.keys())[0]
        files[idd] = [d + '/' + f for f in fna.values()] # add .fna to file list
        
        found = find_files(d, idd)
        files[idd].append(d + '/' + found[0][idd])
        files[idd].append(d + '/' + found[1][idd])
    
for idd in files.keys():
    if not os.path.exists(os.path.join(pulpyd, 'genomes/')):
        os.makedirs(os.path.join(pulpyd, 'genomes/'))
    if not os.path.exists(os.path.join(pulpyd, 'proteins/')):
        os.makedirs(os.path.join(pulpyd, 'proteins/'))
    
    arr = files[idd]
    if arr[0].endswith('.fna.gz'):
        shutil.copy(arr[0], os.path.join(pulpyd, 'genomes/' + idd + '_genomic.fna.gz'))
    else: # if a .fna file was provided, then compress to .fna.gz
        with open(arr[0], 'rb') as f_in:
            with gzip.open(os.path.join(pulpyd, 'genomes/' + idd + '_genomic.fna.gz'), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    shutil.copy(arr[1], os.path.join(pulpyd, 'proteins/' + idd + '_protein.faa'))
    shutil.copy(arr[2], os.path.join(pulpyd, 'proteins/' + idd + '.gff'))